In [ ]:
# import json
# import numpy as np
# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return json.JSONEncoder.default(self, obj)

In [ ]:
# var_values = dict()
# var_values["hi"] = {"hello":np.ones((2,2))}
# _json_str = json.dumps(var_values, cls = NumpyEncoder)
# with open("some.json",'w') as f:
#     json.dump(_json_str,f)
    
    

In [1]:
import json
import numpy as np
import os
# fname = os.path.join('C:','Users','86165','Desktop','tank_admm.json')
#fname = "tank_admm.json"
fname = os.path.join('json_data','tank_admm_100_1w.json')
with open(fname) as f_obj:
    restore = json.load(f_obj)
json_data = json.loads(restore) # the data of dict data
W1 = json_data["w1"]
W2 = json_data["w2"]
b1 = json_data["b1"]
b2 = json_data["b2"]

W1_mean = np.zeros((4,10))
b1_mean = np.zeros(10)
W2_mean = np.zeros((10,2))
b2_mean = np.zeros(2)

ns = len(W1)
for x in W1.values():
    W1_mean += np.array(x)
W1_mean = W1_mean/ns

for x in W2.values():
    W2_mean += np.array(x)
W2_mean = W2_mean/ns

for x in b1.values():
    b1_mean += np.array(x)
b1_mean = b1_mean/ns

for x in b2.values():
    b2_mean += np.array(x)
b2_mean = b2_mean/ns
    
W1["Scenario0"] = W1_mean
W2["Scenario0"] = W2_mean
b1["Scenario0"] = b1_mean
b2["Scenario0"] = b2_mean
    
# compute the norm:
norm = np.zeros(ns+1)
for i in range(1,ns):
    norm[i-1] = np.linalg.norm(W1["Scenario"+str(i)] - W1["Scenario0"])+np.linalg.norm(W2["Scenario"+str(i)] - W2["Scenario0"])\
        +np.linalg.norm(b1["Scenario"+str(i)] - b1["Scenario0"]) + np.linalg.norm(b2["Scenario"+str(i)] - b2["Scenario0"])

print(ns)
print(norm)

40
[1.90797409 3.57364279 2.55066262 2.07964614 2.32537998 1.34707323
 3.21405135 2.11370621 3.06975542 2.3738232  3.89601345 2.74069575
 3.27128843 2.50458839 3.56249784 2.46763733 2.61383006 2.48014868
 3.07275039 4.27269055 3.3830601  3.7909851  3.08525186 2.20631482
 3.32719718 3.24436253 2.03385777 4.51210992 1.92377529 3.36178495
 3.34373963 3.149271   2.32392405 3.17790184 2.08800532 6.12332036
 3.10034207 2.63562527 1.87527235 0.         0.        ]


In [ ]:
# sn = 8

# import numpy as np
# W_1 = np.array(W1["Scenario"+str(sn)])
# W_2 = np.array(W2["Scenario"+str(sn)])
# b_1 = np.array(b1["Scenario"+str(sn)])
# b_2 = np.array(b2["Scenario"+str(sn)])
# def opt(x):
#     global W_1, b_1, W_2, b_2
#     hid1 =  np.tanh(W_1.T@x+b_1)
#     out = np.tanh(W_2.T@hid1+b_2)
#     return out

In [2]:
import h5py
f = h5py.File("test_data_1k.jld","r")
z1_test = f["z1"]
z2_test = f["z2"]
z3_test = f["z3"]
z4_test = f["z4"]

N = 20
h = 3
xs = [14, 14, 14.2, 21.3]
us = [43.4, 35.4]
vmin = np.array([-43.4, -35.4])
vmax = np.array([16.6, 24.6])
zmin = [-6.5, -6.5, -10.7, -16.8]
zmax = [14, 14, 13.8, 6.7]
g = 981
A = [50.27, 50.27, 28.27, 28.27]
a = [0.233, 0.242, 0.127, 0.127]
gama = [0.4, 0.4]
N_sim = 500
n_vali = 500
ntest = 1000
# np.random.seed(1)
# z1_test = np.random.uniform(zmin[0],zmax[0],size = ntest)
# z2_test = np.random.uniform(zmin[1],zmax[1],ntest)
# z3_test = np.random.uniform(zmin[2],zmax[2],ntest)
# z4_test = np.random.uniform(zmin[3],zmax[3],ntest)
np.random.seed(0)
z1_vali = np.random.uniform(zmin[0],zmax[0],size = ntest)
z2_vali = np.random.uniform(zmin[1],zmax[1],ntest)
z3_vali = np.random.uniform(zmin[2],zmax[2],ntest)
z4_vali = np.random.uniform(zmin[3],zmax[3],ntest)


def denormalizeV(v):
    return (vmax + vmin)/2 + (vmax- vmin)/2 *v

def step_model(z,v):
    zold = z.copy()
    z1 = zold[0] + h*(-a[0]/A[0]*np.sqrt(2*g*(zold[0]+xs[0])) + a[2]/A[0]*np.sqrt(2*g*(zold[2]+xs[2])) + gama[0]/A[0]*(v[0]+us[0]) )
    z2 = zold[1] + h*(-a[1]/A[1]*np.sqrt(2*g*(zold[1]+xs[1])) + a[3]/A[1]*np.sqrt(2*g*(zold[3]+xs[3])) + gama[1]/A[1]*(v[1]+us[1]) )
    z3 = zold[2] + h*(-a[2]/A[2]*np.sqrt(2*g*(zold[2]+xs[2])) + (1-gama[1])/A[2]*(v[1]+us[1]) )
    z4 = zold[3] + h*(-a[3]/A[3]*np.sqrt(2*g*(zold[3]+xs[3])) + (1-gama[0])/A[3]*(v[0]+us[0]) )
    return np.array([z1,z2,z3,z4])



# cost_opt_test = np.array([])
# vio_opt_test = np.array([])
# vio_percent_opt_test = np.array([])

# for i in range(ntest):
#     v_trial = np.zeros((2,N_sim))
#     z_trial = np.zeros((4,N_sim+1))
#     z_trial[:,0] = np.array([z1_test[i],z2_test[i],z3_test[i],z4_test[i]])
#     for t in range(N_sim):
#         v_trial[:,t] = denormalizeV(opt(z_trial[:,t]))
#         z_trial[:,t+1] = step_model(z_trial[:,t],v_trial[:,t])
#     cost_opt_test = np.append(cost_opt_test, np.sum(z_trial[0,1:N_sim+1]**2 +  z_trial[1,1:N_sim+1]**2) \
#               + np.sum(0.01*v_trial[0,:]**2+0.01*v_trial[1,:]**2))
#     vio_opt_test = np.append(vio_opt_test, np.maximum(np.max(z_trial[0,:] - zmax[0]),0) + np.maximum(np.max(zmin[0]-z_trial[0,:]),0) + \
#              np.maximum(np.max(z_trial[1,:]-zmax[1]),0) + np.maximum(np.max(zmin[1]-z_trial[1,:]),0) +\
#              np.maximum(np.max(z_trial[2,:]-zmax[2]),0) + np.maximum(np.max(zmin[2]-z_trial[2,:]),0) +\
#              np.maximum(np.max(z_trial[3,:]-zmax[3]),0) + np.maximum(np.max(zmin[3]-z_trial[3,:]),0))
#     if vio_opt_test[i] >= 1e-3:
#         vio_percent_opt_test = np.append(vio_percent_opt_test,1)
#     else:
#         vio_percent_opt_test = np.append(vio_percent_opt_test,0)
        
# print("Ipopt mean cost test: ", np.mean(cost_opt_test),"\n")
# print("Ipopt vio test: ", np.max(vio_opt_test),"\n")
# print("Ipopt vio percent in test: ", np.sum(vio_percent_opt_test)/ntest,"\n")


In [3]:
SN = ns
cost_mat = np.zeros(SN+1)
vio_max = np.zeros(SN+1)
vio_per = np.zeros(SN+1)

for sn in range(SN+1):
    W_1 = np.array(W1["Scenario"+str(sn)])
    W_2 = np.array(W2["Scenario"+str(sn)])
    b_1 = np.array(b1["Scenario"+str(sn)])
    b_2 = np.array(b2["Scenario"+str(sn)])
    def opt(x):
        hid1 =  np.tanh(W_1.T@x+b_1)
        out = np.tanh(W_2.T@hid1+b_2)
        return out
    
    cost_opt_vali = np.array([])
    vio_opt_vali = np.array([])
    vio_percent_opt_vali = np.array([])

    for i in range(n_vali):
        v_trial = np.zeros((2,N_sim))
        z_trial = np.zeros((4,N_sim+1))
        z_trial[:,0] = np.array([z1_vali[i],z2_vali[i],z3_vali[i],z4_vali[i]])
        for t in range(N_sim):
            v_trial[:,t] = denormalizeV(opt(z_trial[:,t]))
            z_trial[:,t+1] = step_model(z_trial[:,t],v_trial[:,t])
        cost_opt_vali = np.append(cost_opt_vali, np.sum(z_trial[0,1:N_sim+1]**2 +  z_trial[1,1:N_sim+1]**2) \
                  + np.sum(0.01*v_trial[0,:]**2+0.01*v_trial[1,:]**2))
        vio_opt_vali = np.append(vio_opt_vali, np.maximum(np.max(z_trial[0,:] - zmax[0]),0) + np.maximum(np.max(zmin[0]-z_trial[0,:]),0) + \
                 np.maximum(np.max(z_trial[1,:]-zmax[1]),0) + np.maximum(np.max(zmin[1]-z_trial[1,:]),0) +\
                 np.maximum(np.max(z_trial[2,:]-zmax[2]),0) + np.maximum(np.max(zmin[2]-z_trial[2,:]),0) +\
                 np.maximum(np.max(z_trial[3,:]-zmax[3]),0) + np.maximum(np.max(zmin[3]-z_trial[3,:]),0))
        if vio_opt_vali[i] >= 1e-3:
            vio_percent_opt_vali = np.append(vio_percent_opt_vali,1)
        else:
            vio_percent_opt_vali = np.append(vio_percent_opt_vali,0)
            
    cost_mat[sn] = np.mean(cost_opt_vali)
    vio_max[sn] = np.max(vio_opt_vali)
    vio_per[sn] = np.sum(vio_percent_opt_vali)/n_vali
    
print(cost_mat)

print(vio_max)

print(vio_per)



[857.15042136 399.123181   398.49734894 397.47670739 403.65320144
 395.95949908 400.04175958 395.48215214 401.54004474 402.549485
 397.62823432 396.42684796 397.5281278  397.61075515 400.7507265
 398.04564712 400.85521749 396.17583259 396.79825147 402.28250525
 394.886586   401.04312772 398.30580919 397.82438906 399.04973562
 399.6551417  398.86280996 400.950842   396.23822155 400.37423842
 396.96369017 402.16546312 395.55444328 404.00679276 398.48483216
 420.31373289 395.19148581 397.68902885 399.02376874 399.09085573
 405.78241087]
[ 6.36288667  1.60542449  1.33832238  1.68869878  2.67463628  2.16795951
  1.82901706  1.72948378  1.21172974  1.41566888  1.63958087  1.11343579
  1.22161624  0.88146401  1.23769269  0.92091381  2.60110419  1.61346466
  1.10474841  3.351919    1.419719    1.73115997  0.97347391  1.22978903
  1.97672081  1.24051176  0.78799502  1.74751776  1.51953417  1.2532766
  1.44503606  0.5771426   2.97077475  1.62171154  3.47473766 12.92809764
  1.07624644  1.2505472

In [4]:
sn_min = np.argmin(vio_per[0:ns+1])
print(sn_min)

31


In [5]:

W_1 = np.array(W1["Scenario"+str(sn_min)])
W_2 = np.array(W2["Scenario"+str(sn_min)])
b_1 = np.array(b1["Scenario"+str(sn_min)])
b_2 = np.array(b2["Scenario"+str(sn_min)])
def opt(x):
    hid1 =  np.tanh(W_1.T@x+b_1)
    out = np.tanh(W_2.T@hid1+b_2)
    return out

cost_opt_test = np.array([])
vio_opt_test = np.array([])
vio_percent_opt_test = np.array([])

for i in range(ntest):
    v_trial = np.zeros((2,N_sim))
    z_trial = np.zeros((4,N_sim+1))
    z_trial[:,0] = np.array([z1_test[i],z2_test[i],z3_test[i],z4_test[i]])
    for t in range(N_sim):
        v_trial[:,t] = denormalizeV(opt(z_trial[:,t]))
        z_trial[:,t+1] = step_model(z_trial[:,t],v_trial[:,t])
    cost_opt_test = np.append(cost_opt_test, np.sum(z_trial[0,1:N_sim+1]**2 +  z_trial[1,1:N_sim+1]**2) \
              + np.sum(0.01*v_trial[0,:]**2+0.01*v_trial[1,:]**2))
    vio_opt_test = np.append(vio_opt_test, np.maximum(np.max(z_trial[0,:] - zmax[0]),0) + np.maximum(np.max(zmin[0]-z_trial[0,:]),0) + \
             np.maximum(np.max(z_trial[1,:]-zmax[1]),0) + np.maximum(np.max(zmin[1]-z_trial[1,:]),0) +\
             np.maximum(np.max(z_trial[2,:]-zmax[2]),0) + np.maximum(np.max(zmin[2]-z_trial[2,:]),0) +\
             np.maximum(np.max(z_trial[3,:]-zmax[3]),0) + np.maximum(np.max(zmin[3]-z_trial[3,:]),0))
    if vio_opt_test[i] >= 1e-3:
        vio_percent_opt_test = np.append(vio_percent_opt_test,1)
    else:
        vio_percent_opt_test = np.append(vio_percent_opt_test,0)
print("Ipopt mean cost test: ", np.mean(cost_opt_test),"\n")
print("Ipopt vio test: ", np.max(vio_opt_test),"\n")
print("Ipopt vio percent in test: ", np.mean(vio_percent_opt_test),"\n")          

Ipopt mean cost test:  387.34534248617786 

Ipopt vio test:  0.8269018841974436 

Ipopt vio percent in test:  0.011 



In [ ]:
print(np.around(W1["Scenario5"],10))

In [ ]:
W2["Scenario5"]

In [ ]:
b1["Scenario5"]

In [ ]:
b2["Scenario5"]

In [ ]:
import json
 import numpy as np
with open("some.json") as f_obj:
    restore = json.load(f_obj)
print(restore)
dumps = json.loads(restore)
x = dumps["hi"]
for y in x.values():
    print(np.asarray(y))
# _hi = np.asarray(dumps["hi"]["hello"])


# print(_hi)  